In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.preprocessing import Binarizer
from sklearn.compose import make_column_transformer

# ORIG 

In [3]:
headers_orig = open('headers_orig.txt', 'r')
headers_orig = headers_orig.read().strip().split('\n')
orig = pd.read_csv("sample_orig_2018.txt", index_col=False, names=headers_orig, sep="|")
orig.head()

,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,...,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator
0,784,201803,9,204802,49180.0,0,1,I,75,39,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
1,693,201803,N,203302,24340.0,0,1,P,80,41,...,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N
2,731,201803,9,204802,NaN,0,1,P,45,37,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
3,763,201802,Y,204801,NaN,30,1,P,95,34,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
4,755,201803,9,204802,49700.0,0,1,P,80,33,...,360,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N


# SVCG

In [3]:
headers_svcg = open('headers_svcg.txt', 'r')
headers_svcg = headers_svcg.read().strip().split('\n')
svcg = pd.read_csv("sample_svcg_2018.txt", index_col=False, names=headers_svcg, sep="|")
svcg.head()

c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (3,7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Loan Sequence Number,Monthly Reporting Period,Current Actual UPB,Current Loan Delinquency Status,Loan Age,Remaining Months to Legal Maturity,Repurchase Flag,Modification Flag,Zero Balance Code,Zero Balance Effective Date,...,Miscellaneous Expenses,Actual Loss Calculation,Modification Cost,Step Modification Flag,Deferred Payment Plan,Estimated Loan-to-Value (ELTV),Zero Balance Removal UPB,Delinquent Accrued Interest,Delinquency Due to Disaster,Borrower Assistance Status Code
0,F18Q10000027,201802,74000.0,0,0,360,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,75.0,NaN,NaN,NaN,NaN
1,F18Q10000027,201803,74000.0,0,1,359,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,73.0,NaN,NaN,NaN,NaN
2,F18Q10000027,201804,73000.0,0,2,358,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN
3,F18Q10000027,201805,73000.0,0,3,357,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN
4,F18Q10000027,201806,73000.0,0,4,356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN


# Join SVCG+Orig -> new_sample.csv

In [4]:
new_svcg = svcg.head(50000)
new_svcg = new_svcg.join(orig.set_index("Loan Sequence Number"), on="Loan Sequence Number", how="left")
new_svcg.to_csv('new_sample.csv', index=False) 

In [5]:
svcg = new_svcg

In [4]:
# zero_balance_codes = {1: 0, 2: 0, 3: 1, 6: 1, 9: 1, 15: 0, np.nan:0}
# svcg['Zero Balance Code'] = svcg['Zero Balance Code'].replace(zero_balance_codes)
# svcg['Current Loan Delinquency Status'] = svcg['Current Loan Delinquency Status'].replace({'R': 100, 'XX': 100}).astype(int)

In [5]:
# bi_1 = Binarizer(threshold=2)
# ct_1 = make_column_transformer(
#     (bi, ['Current Loan Delinquency Status']),
#     remainder='passthrough'
# )

# new_svcg = pd.DataFrame(ct.fit_transform(svcg))
# new_svcg

In [6]:
def current_loan_helper(val):
    try:
        if int(val) >= 3:
            return 1
        else:
            return 0
    except:
        return 1
    
def zero_balance_helper(val):
    target = [3,6,9]
    if val in target:
        return 1
    else:
        return 0
    
def default_helper(val):
    if val > 0:
        return 1
    else:
        return 0

In [7]:
svcg['Zero Balance Code'] = svcg['Zero Balance Code'].apply(zero_balance_helper)

In [8]:
svcg['Current Loan Delinquency Status'] = svcg['Current Loan Delinquency Status'].apply(current_loan_helper)

In [9]:
svcg = svcg.assign(**{'Default': svcg['Current Loan Delinquency Status'] + svcg['Zero Balance Code']})
svcg['Default'] = svcg['Default'].apply(default_helper)

In [10]:
svcg

,Loan Sequence Number,Monthly Reporting Period,Current Actual UPB,Current Loan Delinquency Status,Loan Age,Remaining Months to Legal Maturity,Repurchase Flag,Modification Flag,Zero Balance Code,Zero Balance Effective Date,...,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator,Default
0,F18Q10000027,201802,74000.0,0,0,360,NaN,NaN,0,NaN,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
1,F18Q10000027,201803,74000.0,0,1,359,NaN,NaN,0,NaN,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
2,F18Q10000027,201804,73000.0,0,2,358,NaN,NaN,0,NaN,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
3,F18Q10000027,201805,73000.0,0,3,357,NaN,NaN,0,NaN,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
4,F18Q10000027,201806,73000.0,0,4,356,NaN,NaN,0,NaN,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,F18Q10038568,201807,0.0,0,5,355,N,NaN,0,201807.0,...,1,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,H,NaN,2,N,0
49996,F18Q10038594,201802,249000.0,0,0,360,NaN,NaN,0,NaN,...,2,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,9,NaN,2,N,0
49997,F18Q10038594,201803,249000.0,0,1,359,NaN,NaN,0,NaN,...,2,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,9,NaN,2,N,0
49998,F18Q10038594,201804,248000.0,0,2,358,NaN,NaN,0,NaN,...,2,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,9,NaN,2,N,0


In [25]:
cols = orig.columns.to_list()
cols.append("Default")
svcg = svcg.filter(items=cols)
svcg = svcg.drop_duplicates()

In [26]:
svcg.dtypes

Credit Score                                                      int64
First Payment Date                                                int64
First Time Homebuyer Flag                                        object
Maturity Date                                                     int64
Metropolitan Statistical Area (MSA) Or Metropolitan Division    float64
Mortgage Insurance Percentage (MI %)                              int64
Number of Units                                                   int64
Occupancy Status                                                 object
Original Combined Loan-to-Value (CLTV)                            int64
Original Debt-to-Income (DTI) Ratio                               int64
Original UPB                                                      int64
Original Loan-to-Value (LTV)                                      int64
Original Interest Rate                                          float64
Channel                                                         

In [1]:
enc = OneHotEncoder()

NameError: name 'OneHotEncoder' is not defined

In [53]:
from sklearn.preprocessing import OneHotEncoder
svcg['Property Valuation Method'].value_counts().sort_index()

1      60
2    1611
3      23
9       5
Name: Property Valuation Method, dtype: int64

In [19]:
oneHotSvcg = pd.get_dummies(svcg,drop_first=True)

In [21]:
X = oneHotSvcg.drop('Default', axis=1)
y = oneHotSvcg['Default']

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [24]:
oneHotSvcg

,Credit Score,First Payment Date,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,Original UPB,Original Loan-to-Value (LTV),...,Pre-HARP Loan Sequence Number_F07Q10331087,Pre-HARP Loan Sequence Number_F07Q20217062,Pre-HARP Loan Sequence Number_F07Q20322596,Pre-HARP Loan Sequence Number_F07Q30068951,Pre-HARP Loan Sequence Number_F07Q40019430,Pre-HARP Loan Sequence Number_F07Q40390679,Pre-HARP Loan Sequence Number_F08Q10300997,Pre-HARP Loan Sequence Number_F08Q10505823,Pre-HARP Loan Sequence Number_F09Q20282040,Program Indicator_H
0,784,201803,204802,49180.0,0,1,75,39,74000,75,...,0,0,0,0,0,0,0,0,0,0
22,693,201803,203302,24340.0,0,1,80,41,132000,80,...,0,0,0,0,0,0,0,0,0,0
49,731,201803,204802,NaN,0,1,45,37,175000,45,...,0,0,0,0,0,0,0,0,0,0
75,763,201802,204801,NaN,30,1,95,34,27000,95,...,0,0,0,0,0,0,0,0,0,0
110,755,201803,204802,49700.0,0,1,80,33,168000,80,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49885,787,201803,204802,10900.0,12,1,85,42,85000,85,...,0,0,0,0,0,0,0,0,0,0
49920,713,201803,204802,29404.0,0,1,80,44,284000,80,...,0,0,0,0,0,0,0,0,0,0
49955,735,201803,204802,33124.0,30,1,95,50,341000,95,...,0,0,0,0,0,0,0,0,0,0
49990,792,201803,204802,35614.0,12,1,85,34,336000,85,...,0,0,0,0,0,0,0,0,0,1


In [23]:
reg = LinearRegression().fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [43]:
svcg

,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,Original UPB,Original Loan-to-Value (LTV),Original Interest Rate,Channel,Prepayment Penalty Mortgage (PPM) Flag,Amortization Type (Formerly Product Type),Property State,Property Type,Postal Code,Loan Sequence Number,Loan Purpose,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator,Default
0,784,201803,9,204802,49180.0,0,1,I,75,39,74000,75,4.625,R,N,FRM,NC,PU,27100,F18Q10000027,C,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
22,693,201803,N,203302,24340.0,0,1,P,80,41,132000,80,3.250,R,N,FRM,MI,SF,49300,F18Q10000052,P,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N,0
49,731,201803,9,204802,NaN,0,1,P,45,37,175000,45,3.750,R,N,FRM,MN,SF,56500,F18Q10000083,C,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
75,763,201802,Y,204801,NaN,30,1,P,95,34,27000,95,4.250,R,N,FRM,WI,SF,54500,F18Q10000116,P,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
110,755,201803,9,204802,49700.0,0,1,P,80,33,168000,80,3.875,R,N,FRM,CA,PU,95900,F18Q10000139,C,360,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49885,787,201803,9,204802,10900.0,12,1,P,85,42,85000,85,4.625,R,N,FRM,NJ,SF,8800,F18Q10038508,C,360,1,"WELLS FARGO BANK, N.A.","WELLS FARGO BANK, N.A.",NaN,NaN,9,NaN,2,N,0
49920,713,201803,Y,204802,29404.0,0,1,P,80,44,284000,80,4.375,B,N,FRM,IL,SF,60000,F18Q10038539,P,360,2,QUICKEN LOANS INC.,NEW RESIDENTIAL MORTGAGE LLC,NaN,NaN,9,NaN,2,N,0
49955,735,201803,Y,204802,33124.0,30,1,P,95,50,341000,95,4.625,B,N,FRM,FL,SF,33100,F18Q10038546,P,360,2,QUICKEN LOANS INC.,NEW RESIDENTIAL MORTGAGE LLC,NaN,NaN,9,NaN,2,N,0
49990,792,201803,Y,204802,35614.0,12,1,P,85,34,336000,85,4.625,B,N,FRM,NJ,CO,8800,F18Q10038568,P,360,1,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,H,NaN,2,N,0


In [23]:
svcg.columns

Index(['Loan Sequence Number', 'Monthly Reporting Period',
       'Current Actual UPB', 'Current Loan Delinquency Status', 'Loan Age',
       'Remaining Months to Legal Maturity', 'Repurchase Flag',
       'Modification Flag', 'Zero Balance Code', 'Zero Balance Effective Date',
       'Current Interest Rate', 'Current Deferred UPB',
       'Due Date of Last Paid Installment (DDLPI)', 'MI Recoveries',
       'Net Sales Proceeds', 'Non MI Recoveries', 'Expenses', 'Legal Costs',
       'Maintenance and Preservation Costs', 'Taxes and Insurance',
       'Miscellaneous Expenses', 'Actual Loss Calculation',
       'Modification Cost', 'Step Modification Flag', 'Deferred Payment Plan',
       'Estimated Loan-to-Value (ELTV)', 'Zero Balance Removal UPB',
       'Delinquent Accrued Interest', 'Delinquency Due to Disaster',
       'Borrower Assistance Status Code', 'Credit Score', 'First Payment Date',
       'First Time Homebuyer Flag', 'Maturity Date',
       'Metropolitan Statistical Area (MS

To Delete:
Loan Sequence Number
Maybe Delete:
Monthly Reporting Period

In [25]:
pd.set_option('display.max_columns', None)
svcg

,Loan Sequence Number,Monthly Reporting Period,Current Actual UPB,Current Loan Delinquency Status,Loan Age,Remaining Months to Legal Maturity,Repurchase Flag,Modification Flag,Zero Balance Code,Zero Balance Effective Date,Current Interest Rate,Current Deferred UPB,Due Date of Last Paid Installment (DDLPI),MI Recoveries,Net Sales Proceeds,Non MI Recoveries,Expenses,Legal Costs,Maintenance and Preservation Costs,Taxes and Insurance,Miscellaneous Expenses,Actual Loss Calculation,Modification Cost,Step Modification Flag,Deferred Payment Plan,Estimated Loan-to-Value (ELTV),Zero Balance Removal UPB,Delinquent Accrued Interest,Delinquency Due to Disaster,Borrower Assistance Status Code,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,Original UPB,Original Loan-to-Value (LTV),Original Interest Rate,Channel,Prepayment Penalty Mortgage (PPM) Flag,Amortization Type (Formerly Product Type),Property State,Property Type,Postal Code,Loan Purpose,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator,Default
0,F18Q10000027,201802,74000.0,0,0,360,NaN,NaN,0,NaN,4.625,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,NaN,NaN,NaN,NaN,784,201803,9,204802,49180.0,0,1,I,75,39,74000,75,4.625,R,N,FRM,NC,PU,27100,C,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
1,F18Q10000027,201803,74000.0,0,1,359,NaN,NaN,0,NaN,4.625,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,NaN,NaN,NaN,NaN,784,201803,9,204802,49180.0,0,1,I,75,39,74000,75,4.625,R,N,FRM,NC,PU,27100,C,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
2,F18Q10000027,201804,73000.0,0,2,358,NaN,NaN,0,NaN,4.625,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN,784,201803,9,204802,49180.0,0,1,I,75,39,74000,75,4.625,R,N,FRM,NC,PU,27100,C,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
3,F18Q10000027,201805,73000.0,0,3,357,NaN,NaN,0,NaN,4.625,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN,784,201803,9,204802,49180.0,0,1,I,75,39,74000,75,4.625,R,N,FRM,NC,PU,27100,C,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
4,F18Q10000027,201806,73000.0,0,4,356,NaN,NaN,0,NaN,4.625,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN,784,201803,9,204802,49180.0,0,1,I,75,39,74000,75,4.625,R,N,FRM,NC,PU,27100,C,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,F18Q10038568,201807,0.0,0,5,355,N,NaN,0,201807.0,4.625,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,334000.0,NaN,NaN,NaN,792,201803,Y,204802,35614.0,12,1,P,85,34,336000,85,4.625,B,N,FRM,NJ,CO,8800,P,360,1,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,H,NaN,2,N,0
49996,F18Q10038594,201802,249000.0,0,0,360,NaN,NaN,0,NaN,4.990,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,NaN,NaN,NaN,NaN,684,201803,9,204802,10420.0,0,1,P,80,30,249000,80,4.990,R,N,FRM,OH,SF,44300,C,360,2,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,9,NaN,2,N,0
49997,F18Q10038594,201803,249000.0,0,1,359,NaN,NaN,0,NaN,4.990,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,NaN,NaN,NaN,NaN,684,201803,9,204802,10420.0,0,1,P,80,30,249000,80,4.990,R,N,FRM,OH,SF,44300,C,360,2,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,9,NaN,2,N,0
49998,F18Q10038594,201804,248000.0,0,2,358,NaN,NaN,0,NaN,4.990,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.0,NaN,NaN,NaN,NaN,684,201803,9,204802,10420.0,0,1,P,80

['Credit Score',
 'First Payment Date',
 'First Time Homebuyer Flag',
 'Maturity Date',
 'Metropolitan Statistical Area (MSA) Or Metropolitan Division',
 'Mortgage Insurance Percentage (MI %)',
 'Number of Units',
 'Occupancy Status',
 'Original Combined Loan-to-Value (CLTV)',
 'Original Debt-to-Income (DTI) Ratio',
 'Original UPB',
 'Original Loan-to-Value (LTV)',
 'Original Interest Rate',
 'Channel',
 'Prepayment Penalty Mortgage (PPM) Flag',
 'Amortization Type (Formerly Product Type)',
 'Property State',
 'Property Type',
 'Postal Code',
 'Loan Sequence Number',
 'Loan Purpose',
 'Original Loan Term',
 'Number of Borrowers',
 'Seller Name',
 'Servicer Name',
 'Super Conforming Flag',
 'Pre-HARP Loan Sequence Number',
 'Program Indicator',
 'HARP Indicator',
 'Property Valuation Method',
 'Interest Only (I/O) Indicator',
 'Default']

,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,Original UPB,Original Loan-to-Value (LTV),Original Interest Rate,Channel,Prepayment Penalty Mortgage (PPM) Flag,Amortization Type (Formerly Product Type),Property State,Property Type,Postal Code,Loan Sequence Number,Loan Purpose,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator,Default
0,784,201803,9,204802,49180.0,0,1,I,75,39,74000,75,4.625,R,N,FRM,NC,PU,27100,F18Q10000027,C,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
1,784,201803,9,204802,49180.0,0,1,I,75,39,74000,75,4.625,R,N,FRM,NC,PU,27100,F18Q10000027,C,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
2,784,201803,9,204802,49180.0,0,1,I,75,39,74000,75,4.625,R,N,FRM,NC,PU,27100,F18Q10000027,C,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
3,784,201803,9,204802,49180.0,0,1,I,75,39,74000,75,4.625,R,N,FRM,NC,PU,27100,F18Q10000027,C,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
4,784,201803,9,204802,49180.0,0,1,I,75,39,74000,75,4.625,R,N,FRM,NC,PU,27100,F18Q10000027,C,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,792,201803,Y,204802,35614.0,12,1,P,85,34,336000,85,4.625,B,N,FRM,NJ,CO,8800,F18Q10038568,P,360,1,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,H,NaN,2,N,0
49996,684,201803,9,204802,10420.0,0,1,P,80,30,249000,80,4.990,R,N,FRM,OH,SF,44300,F18Q10038594,C,360,2,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,9,NaN,2,N,0
49997,684,201803,9,204802,10420.0,0,1,P,80,30,249000,80,4.990,R,N,FRM,OH,SF,44300,F18Q10038594,C,360,2,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,9,NaN,2,N,0
49998,684,201803,9,204802,10420.0,0,1,P,80,30,249000,80,4.990,R,N,FRM,OH,SF,44300,F18Q10038594,C,360,2,QUICKEN LOANS INC.,QUICKEN LOANS INC.,NaN,NaN,9,NaN,2,N,0
